# Assignment 3 Pattern miner

Vicente Alejandro Lomellín Ibarra A01175933

Luis Martín Fregoso Aparicio A01339273 

Robertha Isabela Rosas Leal A01196424

In [1]:
!pip install PBC4cip

In [2]:
import os
import argparse
import numpy as np
import pandas as pd
import random

from tqdm import tqdm, trange
from PBC4cip import *
# from PBC4cip import MaximalPatternsGlobalFilter
import random

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import ExtraTreesClassifier
# from PBC4cip_master.PBC4cip1.core import *


import matplotlib.pyplot as plt

## Initialize data

** universities.csv file is required for the test functions

In [3]:
def import_data(csv_data):
  if isinstance(csv_data,str):
    data = pd.read_csv(csv_data, encoding= 'latin-1', index_col= 'University')
  else:
    data = csv_data # If the given data is a pandas DataFrame, pass directly
  predictors = data.iloc[0:data.shape[0],:-1].values
  targets = data.iloc[0:data.shape[0],-1].values
  return predictors, targets, data

In [4]:
def test_import_data():
  df = pd.DataFrame({'Dummy1': list(range(0,10)), 'Dummy2': list(range(10,20)),'class': [0,1,1,0,1,0,1,0,1,0]}, index = list(range(10)))
  X, y = import_data(df)
  assert y.shape == (10,)
  assert X.shape == (10,2)
  assert isinstance(X,np.ndarray)
  assert all(X[0]) == all([0,10]) 
  assert y[0] == 0

In [5]:
def split_data(predictors, target):
  indices = list(range(len(predictors)))
  split = int(np.floor(0.3 * len(predictors)))
  np.random.seed(0)
  np.random.shuffle(indices)

  train_idx, test_idx = indices[split:], indices[:split]
  train_predictors = []
  train_targets = []

  X_train, y_train = get_train_data(train_idx,predictors,target)
  X_test, y_test = get_test_data(test_idx,predictors,target)
  
  return X_train, y_train, X_test, y_test

def get_train_data(train_idx,predictors,target):
  X_train = []
  y_train = []

  for i in train_idx:
    X_train.append(predictors[i])
    y_train.append(target[i])
  return X_train, y_train

def get_test_data(test_idx,predictors,target):
  X_test = []
  y_test = []
  for i in test_idx:
    X_test.append(predictors[i])
    y_test.append(target[i])
  return X_test, y_test
  

In [6]:
def test_split_data():
  df = pd.DataFrame({'Dummy1': list(range(0,10)), 'Dummy2': list(range(10,20)),'class': [0,1,1,0,1,0,1,0,1,0]}, index = list(range(10)))
  X, y = import_data(df)
  X_train, y_train, X_test, y_test = split_data(X, y)
  assert len(X_train) == 7 and len(y_train) == 7
  assert len(X_test) == 3 and len(y_test) == 3
  assert all(X_train[0]) == all([ 9, 19]) and all(X_test[0]) == all([ 2, 12])
  assert all(y_train) == all([0, 1, 1, 0, 0, 0, 0]) and all(y_test) == all([1, 1, 1])
  assert isinstance(X_train,list) and isinstance(y_train,list) and isinstance(X_test,list) and isinstance(y_test,list)


def test_get_test_data():
  df = pd.DataFrame({'Dummy1': list(range(0,10)), 'Dummy2': list(range(10,20)),'class': [0,1,1,0,1,0,1,0,1,0]}, index = list(range(10)))
  X, y = import_data(df)
  test_idx = [0,6,8,3]
  X_test, y_test = get_test_data(test_idx,X,y)
  assert len(X_test) == 4 and len(y_test) == 4
  assert isinstance(X_test,list)
  assert isinstance(X_test[0],np.ndarray)
  assert isinstance(y_test[0],np.int64)
  assert all(y_test) == all([0,1,1,0])
  assert all(X_test[0]) == all([0,10])

def test_get_train_data():
  df = pd.DataFrame({'Dummy1': list(range(0,10)), 'Dummy2': list(range(10,20)),'class': [0,1,1,0,1,0,1,0,1,0]}, index = list(range(10)))
  X, y = import_data(df)
  train_idx = [1,2,4,5,7,9]
  X_train, y_train = get_train_data(train_idx,X,y)
  assert len(X_train) == 6 and len(y_train) == 6
  assert isinstance(X_train,list)
  assert isinstance(X_train[0],np.ndarray)
  assert isinstance(y_train[0],np.int64)
  assert all(y_train) == all([1,1,1,0,0,0])
  assert all(X_train[0]) == all([1,11])


In [7]:
def transform_to_DF(train,test,columns):
  train_to_df = pd.DataFrame(train, columns = columns)
  test_to_df = pd.DataFrame(test, columns = columns)
  return train_to_df, test_to_df

In [8]:
def test_transform_to_DF():
  df = pd.DataFrame({'Dummy1': list(range(0,10)), 'Dummy2': list(range(10,20)),'class': [0,1,1,0,1,0,1,0,1,0]}, index = list(range(10)))
  X, y, data = import_data(df)
  test_idx = [1,2,4,5,7,9]
  train_idx = [0,6,8,3]
  X_train, y_train = get_train_data(train_idx,X,y)
  X_test, y_test = get_test_data(test_idx,X,y)
  X_train, X_test = transform_to_DF(X_train,X_test,data.columns[:-1])
  y_train, y_test = transform_to_DF(y_train,y_test,[data.columns[-1]])
  assert X_train.shape == (4,2) and X_test.shape == (6,2)
  assert isinstance(X_train, pd.core.frame.DataFrame) and isinstance(X_test, pd.core.frame.DataFrame)
  assert y_train.shape == (4,1) and y_test.shape == (6,1)
  assert isinstance(y_train, pd.core.frame.DataFrame) and isinstance(y_test, pd.core.frame.DataFrame)
  assert y_train.columns == 'class' and y_test.columns == 'class'
  assert all(X_train.columns) == all(['Dummy1', 'Dummy2']) and all(X_test.columns) == all(['Dummy1', 'Dummy2'])

In [9]:
def load_train_test_data(csv_file):
  predictors, targets, data = import_data(csv_file)
  X_train, y_train, X_test, y_test = split_data(predictors, targets)
  X_train, X_test = transform_to_DF(X_train,X_test,data.columns[:-1])
  y_train, y_test = transform_to_DF(list(np.where(np.asarray(y_train) == 1, 's1_r','s2_r')),list(np.where(np.asarray(y_test)== 1, 's1_r','s2_r')),[data.columns[-1]])

  return X_train, y_train, X_test, y_test

In [10]:
def test_load_train_test_data():
  df = pd.DataFrame({'Dummy1': list(range(0,10)), 'Dummy2': list(range(10,20)),'class': [0,1,1,0,1,0,1,0,1,0]}, index = list(range(10)))
  X_train, y_train, X_test, y_test = load_train_test_data(df)
  
  assert X_train.shape == (7,2) and X_test.shape == (3,2)
  assert y_train.shape == (7,1) and y_test.shape == (3,1)
  assert X_train.values.all() == (np.asarray([[9,19],[1,11],[6,16],[7,17],[3,13],[0,10],[5,15]])).all() and X_test.values.all() == (np.asarray([[2,12],[8,18], [4,14]])).all()
  assert all(y_train) == all(['s2_r', 's1_r', 's1_r', 's2_r', 's2_r', 's2_r', 's2_r']) and all(y_test) == all(['s1_r','s1_r','s1_r'])
  assert all(X_train.columns) == all(['Dummy1', 'Dummy2']) and all(X_test.columns) == all(['Dummy1', 'Dummy2']) and y_train.columns == ['class'] and y_test.columns == ['class']
  
test_load_train_test_data()

## PBC4cip

In [11]:
def PBC4cip_classifier(csv_file, t_count = 1000):
  """ 
  This function extracts the complete patterns from PBC4cip miner. No modifications were done to the source code, since the
  output of the classifier are the Emerging Patterns that were mined. The function is later used to mine the patterns by using the
  internal pattern miner.
  Input: csv file containing the data to be used, and tree count to be used for the Decision tree construction and pattern mining.
  Output: List containing the found emerging patterns from the data. The output contains to sets of information per mined pattern:
  The found univariate pattern and the support for each class s1_r and s2_r
  """
  X_train, y_train, X_test, y_test = load_train_test_data(csv_file)
  classifier = PBC4cip(filtering=True, tree_count = t_count)
  patterns = classifier.fit(X_train, y_train)
  return patterns


In [12]:
def test_PBC4cip_classifier():
  patterns = PBC4cip_classifier('universities.csv', t_count = 10)
  assert len(patterns) < 300
  assert isinstance(patterns[0].Items,list)
  assert isinstance(patterns[0].Supports,list)
  assert isinstance(p[0].Supports[0],float)
  assert len(patterns[0].Items) >= 1 and len(patterns[0].Supports) == 2

In [13]:
def get_best_patterns(pattern, diff):
  """
  Evaluates the pattern for it to comply with the support difference restrictrion
  Input: Pattern to be evaluated and difference between patterns
  Output: Pattern that complies with restriction or None, to be later removed from list
  """
  if (pattern.Supports[0] - pattern.Supports[1]) >= diff:
    return pattern
  else:
    return None

In [14]:
def test_get_best_patterns():
  all_p = PBC4cip_classifier_RandFeat('universities.csv', t_count= 10)
  best_patterns_first = []
  best_patterns_second = []
  for pattern in all_p:
    best_patterns_first.append(get_best_patterns(pattern[0], diff = 0.01))
    best_patterns_second.append(get_best_patterns(pattern[1], diff = 0.01))
  assert isinstance(best_patterns_first,list)
  assert len(best_patterns_first) == 4 and len(best_patterns_second) == 4 
  assert len(best_patterns_first) - best_patterns_first.count(None) >= 1 and len(best_patterns_second) - best_patterns_second.count(None) >= 1
  best_patterns_first = list(filter(None, best_patterns_first))
  best_patterns_second = list(filter(None, best_patterns_second))
  assert best_patterns_first[0].Supports[0] - best_patterns_first[0].Supports[1] >= 0.01
  assert best_patterns_second[0].Supports[0] - best_patterns_second[0].Supports[1] >= 0.01

In [15]:
def test_evaluate_patterns():
  all_p = PBC4cip_classifier_RandFeat('universities.csv', t_count= 10)
  best_patterns = evaluate_patterns(all_p[0])
  assert isinstance(best_patterns,list)
  assert len(best_patterns) == 2
  assert isinstance(best_patterns[0].Items,list) and isinstance(best_patterns[1].Items,list)
  assert str(bp[0].Items[0]) == 'cit2017 > 90284.000' or str(bp[1].Items[0]) == 'cit2017 > 90284.000'
  assert bp[0].Supports[0] + bp[0].Supports[1] >= 0.3

In [16]:
def get_patterns_PBC4cip_classifier(csv_file, t_count, diff = 0.3):
  all_patterns = PBC4cip_classifier(csv_file, t_count)
  best_patterns = []
  for pattern in all_patterns:
    best_patterns.append(get_best_patterns(pattern, diff))
  best_patterns = list(filter(None, best_patterns))
  return best_patterns

In [17]:
def test_get_patterns_PBC4cip_classifier():
  patterns = get_patterns_PBC4cip_classifier('universities.csv', t_count=10)
  assert isinstance(patterns,list)
  assert len(patterns) >= 3 #should return at least three patterns that complies with restriction
  assert patterns[0].Supports[0] - patterns[0].Supports[1] >= 0.3
  assert patterns[1].Supports[0] - patterns[1].Supports[1] >= 0.3
  assert patterns[2].Supports[0] - patterns[2].Supports[1] >= 0.3
  return patterns


## Random Features
5, 10, 15, and 20 random features

In [18]:
def rand_feature_selection(num_features):
  """
  This function returns a number of indeces to pop out of the data features
  Input: Number of features of the dataset
  Output: List of lists containing indices to remove from original data
  """
  random.seed(1)
  rand_feat_idx5  = random.sample(range(num_features), num_features-5)
  rand_feat_idx10  = random.sample(range(num_features), num_features-10)
  rand_feat_idx15  = random.sample(range(num_features), num_features-15)
  rand_feat_idx20  = random.sample(range(num_features), num_features-20)

  
  return [rand_feat_idx5,rand_feat_idx10,rand_feat_idx15,rand_feat_idx20]

In [19]:
def test_rand_feature_selection():
  feat_list = rand_feature_selection(40)
  assert isinstance(feat_list,list)
  assert len(feat_list[0]) == 35
  assert len(feat_list[1]) == 30
  assert len(feat_list[2]) == 25
  assert len(feat_list[3]) == 20


In [20]:
def random_feature_dataframe_generation(feats, X_train, X_test):
  """
  This function remakes the dataframe with the features selected by the random feature selector
  The objective of this function is to restructure the features into the datatype input that PBC4cip uses (pandas DataFrame)
  Input: Features chosen by the random attribute selector, and predictors
  Output: Pandas DataFrame containing the chosen attributes and the predictor's values
  """
  X_train2 = X_train.copy()
  X_test2 = X_test.copy()
  for feat in feats:
    drop_idx = X_train.columns[feat]
    X_train2 = X_train2.drop([drop_idx], axis = 1)
    X_test2 = X_test2.drop([drop_idx], axis = 1)
  return X_train2, X_test2


def get_random_feature_dataframe(csv_file):
  X_train, _, X_test, _ = load_train_test_data(csv_file)
  feat_list = rand_feature_selection(X_train.shape[1])
  X_train_mod = []
  X_test_mod = []
  for feat in feat_list:
    X_tr, X_te = random_feature_dataframe_generation(feat, X_train, X_test)
    X_train_mod.append(X_tr)

  return X_train_mod



In [21]:
def test_random_feature_dataframe_generation():
  X_train, _, X_test, _ = load_train_test_data('universities.csv')
  feats = rand_feature_selection(X_train.shape[1])
  X_train2, X_test2 = random_feature_dataframe_generation(feats[0], X_train, X_test)
  assert X_train2.shape == (140,5)
  assert X_test2.shape == (60,5)
  assert all(X_train2.columns) == all(['intCol2016', 'cit2017', 'pubTJP2018', 'citPP2016', 'pubPA2016'])
  X_train2, X_test2 = random_feature_dataframe_generation(feats[1], X_train, X_test)
  assert X_train2.shape == (140,10)
  assert X_test2.shape == (60,10)
  assert all(X_train2.columns) == all(['intCol2016', 'acaCol2018', 'acaCol2017', 'acaCol2016', 'fwCitImp2018',
       'pubTJP2018', 'pubTJP2016', 'citPP2017', 'citPA2018', 'citPA2016'])
  
def test_get_random_feature_dataframe():
  X_train= get_random_feature_dataframe('universities.csv')
  assert isinstance(X_train,list)
  assert X_train[0].shape == (140,5)
  assert X_train[1].shape == (140,10)
  assert X_train[2].shape == (140,15)
  assert X_train[3].shape == (140,20)

In [22]:
def PBC4cip_classifier_RandFeat(csv_file, t_count):
  """
  This function applies the 
  """
  _, y_train, _, y_test = load_train_test_data(csv_file)
  classifier = PBC4cip(filtering=True,tree_count = t_count)
  all_patterns = []
  y_test_scores = []
  X_train_rf = get_random_feature_dataframe(csv_file)
  for i in range(len(X_train_rf)):
    all_patterns.append(classifier.fit(X_train_rf[i], y_train))
  return all_patterns

In [23]:
def test_PBC4cip_classifier_RandFeat():
  all_p = PBC4cip_classifier_RandFeat('universities.csv', t_count= 10)
  assert isinstance(all_p,list)
  assert len(all_p) == 4
  assert len(all_p[0]) == 247
  assert len(all_p[1]) == 326
  assert all(all_p[0][0].Supports) == all([0.014084507042253521, 0.0])
  assert all(all_p[0][0].Items[0].Feature) == all(('pubTJP2018', 'real'))


In [24]:
def evaluate_patterns(patterns, diff = 0.3):
  """
  This function iterates through the patterns found by each of the random selection that was
  previously performed and sends them to an auxiliar function to evaluate their support.
  The resulting list contains pattern that comply with the restriction of s1_r - s2_r >= 0.3
  Input: Patterns from selected random attributes, and diff, which refers to the difference between both classes supports
  Output: List of patterns that comply with the difference between class support.
  """
  best_patterns = []
  for pattern in patterns:
    best_patterns.append(get_best_patterns(pattern, diff))
  best_patterns = list(filter(None, best_patterns))
  return best_patterns

def get_patterns_RandFeat(csv_file, t_count):
  all_patterns = PBC4cip_classifier_RandFeat(csv_file, t_count)
  get_patterns = []
  for patterns in all_patterns:
    get_patterns.append(evaluate_patterns(patterns, diff = 0.3))
  return get_patterns

In [25]:
def test_evaluate_patterns():
  all_p = PBC4cip_classifier_RandFeat('universities.csv', t_count= 10)
  best_patterns = evaluate_patterns(all_p[0])
  assert isinstance(best_patterns,list)
  assert len(best_patterns) == 2
  assert isinstance(best_patterns[0].Items,list) and isinstance(best_patterns[1].Items,list)
  assert str(bp[0].Items[0]) == 'cit2017 > 90284.000' or str(bp[1].Items[0]) == 'cit2017 > 90284.000'
  assert bp[0].Supports[0] + bp[0].Supports[1] >= 0.3


def test_get_patterns_RandFeat():
  get_patterns = get_patterns_RandFeat('universities.csv', t_count= 10)
  assert isinstance(get_patterns,list)
  assert len(get_patterns) == 4
  assert len(get_patterns[0]) == 2
  assert str(get_patterns[0][0].Items[0]) == 'cit2017 > 90284.000'
  assert get_patterns[0][0].Supports[0] - get_patterns[0][0].Supports[1] >= 0.3 

## Attribute Selection Algorithms

Select K best

Recursive Feature Elimination

Select From Model - Logistic Regression L2 penalty

Feature importance - Extra Trees Classifier

In [26]:
def algorithms_attribute_selection():
  """
  This function returns a number of indeces to pop out of the data features
  Input: Number of features of the dataset
  Output: List of lists containing indices to remove from original data
  """
  random.seed(1)
  f_value_alg = SelectKBest(score_func=f_classif, k=10)

  model = LogisticRegression(solver='lbfgs', random_state = 0)
  rfe = RFE(model, 10)

  model = LogisticRegression(C = 1, penalty = 'l2', solver='lbfgs', random_state= 0)
  sfm = SelectFromModel(model)

  ETC = ExtraTreesClassifier(n_estimators=10)
  
  return [f_value_alg,rfe,sfm,ETC]

In [27]:
def test_algorithm_feature_selection():
  feat_list = algorithms_attribute_selection()
  assert isinstance(feat_list,list)
  assert type(feat_list[0]).__name__ == 'SelectKBest'
  assert type(feat_list[1]).__name__ == 'RFE' 
  assert type(feat_list[2]).__name__ == 'SelectFromModel'
  assert type(feat_list[3]).__name__ == 'ExtraTreesClassifier'
test_algorithm_feature_selection()

In [28]:
def ETC_attribute_selection(feats, X_train):
  ETC_selected_attr_idx = []
  ETC_selected_attr = []
  new_Xtrain = {}
  feat2 = list(feats.copy())

  for i in range(10):
    ETC_selected_attr_idx.append(list(feats).index(max(feat2)))
    feat2.pop(feat2.index(max(feat2)))
  
  for idx in ETC_selected_attr_idx:
    ETC_selected_attr.append(X_train.columns[idx])

  for column in ETC_selected_attr:
    new_Xtrain[column] = X_train[column]
  
  ETC_df = pd.DataFrame.from_dict(new_Xtrain)
  
  return ETC_df


In [57]:
def test_ETC_attribute_selection():
  ETC = ExtraTreesClassifier(n_estimators=10)
  X_tr, y_tr, X_te, y_te = load_train_test_data('universities.csv')
  fit = ETC.fit(X_tr, y_tr)
  features = fit.feature_importances_
  df_ETC = ETC_attribute_selection(features, X_tr)
  assert df_ETC.shape[0] == 140 # complete list of train set
  assert df_ETC.shape[1] == 5
  assert len(df_ETC.columns) == 5
  assert all(df_ETC.columns) == all(['pub2017', 'pub2018', 'pubTJP2018', 'cit2016', 'h5Index', 'pub2016',
       'cit2017', 'authors2017', 'acaCol2017', 'acaCol2016'])
  assert df_ETC['pub2017'][0] == 7617
# (test_ETC_attribute_selection())

In [58]:
def fit_selector(selector, X_train, y_train):
  if type(selector).__name__ == 'ExtraTreesClassifier':
    fit = selector.fit(X_train, y_train)
    return ETC_attribute_selection(fit.feature_importances_, X_train)
  else:
    fit = selector.fit(X_train, y_train)
    return Kbest_RFE_SFM_new_df(X_train, fit.transform(X_train))


def Kbest_RFE_SFM_new_df(X_train, features):
  attributes = []
  for column in X_train.columns:
    attributes.append(return_selected_attributes(X_train, column, features))
  attributes = list(filter(None, attributes))
  if len(attributes) != 5:
    attributes = attributes[0:5]
    features = features[:,0:5]
  new_X_train = pd.DataFrame(features, columns = attributes)

  return new_X_train

def return_selected_attributes(X_tr, column,features):
  if X_tr[column][0] in features[0]:
    return column

In [59]:
def test_fit_selector():
  X_tr, y_tr, _, _ = load_train_test_data('universities.csv')
  feat_list = algorithms_attribute_selection()
  ETC_attribute_selection = fit_selector(feat_list[-1], X_tr, y_tr)
  assert ETC_attribute_selection.shape == (140,5)
  assert isinstance(ETC_attribute_selection,pd.DataFrame)
  Kbest_new_df = fit_selector(feat_list[0], X_tr, y_tr)
  assert Kbest_new_df.shape == (140,5) and isinstance(Kbest_new_df, pd.DataFrame)
  RFE_new_df = fit_selector(feat_list[1], X_tr, y_tr)
  assert RFE_new_df.shape == (140,5) and isinstance(RFE_new_df,pd.DataFrame)
  SFM_new_df = fit_selector(feat_list[2], X_tr, y_tr)
  assert SFM_new_df.shape == (140,5) and isinstance(SFM_new_df,pd.DataFrame)

def test_Kbest_RFE_SFM_new_df():
  X_train, y_train, _, _ = load_train_test_data('universities.csv')
  feat_list = algorithms_attribute_selection()
  fit = feat_list[0].fit(X_train,y_train)
  features = fit.transform(X_train)

  new_df_Kbest = Kbest_RFE_SFM_new_df(X_train, features)
  assert new_df_Kbest.shape == (140,5)
  
  fit = feat_list[1].fit(X_train,y_train)
  features = fit.transform(X_train)
  new_df_RFE = Kbest_RFE_SFM_new_df(X_train, features)
  assert new_df_RFE.shape == (140,5)

  fit = feat_list[2].fit(X_train,y_train)
  features = fit.transform(X_train)
  new_df_SFM = Kbest_RFE_SFM_new_df(X_train, features)
  assert new_df_SFM.shape == (140,5)

  assert isinstance(new_df_Kbest, pd.DataFrame)
  assert isinstance(new_df_RFE, pd.DataFrame)
  assert isinstance(new_df_SFM, pd.DataFrame)

def test_return_selected_attributes():
  X_train, y_train, _, _ = load_train_test_data('universities.csv')
  column = X_train.columns[0]
  features = X_train.values
  col = return_selected_attributes(X_train, column, features)
  assert col == column and col == 'intCol2018'
  
  column = X_train.columns[3]
  col = return_selected_attributes(X_train, column, features)
  assert col == column and col == 'acaCol2018'

  feat_list = algorithms_attribute_selection()
  fit = feat_list[0].fit(X_train,y_train)
  features = fit.transform(X_train)
  column = X_train.columns[2]
  col = return_selected_attributes(X_train, column, features)
  assert col == None

  column = X_train.columns[-1]
  col = return_selected_attributes(X_train, column, features)
  assert col == column and col == 'h5Index'

  attributes = []
  for column in X_train.columns:
    attributes.append(return_selected_attributes(X_train, column, features))
  attributes = list(filter(None, attributes))
  assert len(attributes) == 5

In [60]:
def attribute_selection_df_generator(csv_file):
  X_tr, y_tr, X_te, y_te = load_train_test_data(csv_file)
  selectors = algorithms_attribute_selection()
  new_X_trains = []
  # print(selectors)
  for selector in selectors:
    new_X_trains.append(fit_selector(selector, X_tr, y_tr))
  return new_X_trains

In [61]:
def test_attribute_selection_df_generator():
  new_X_trains = attribute_selection_df_generator('universities.csv')
  assert len(new_X_trains) == 4
  assert new_X_trains[0].shape == (140,10)
  assert new_X_trains[1].shape == (140,10)
  assert new_X_trains[2].shape == (140,10)
  assert isinstance(new_X_trains[0],pd.DataFrame) and isinstance(new_X_trains[1],pd.DataFrame) and isinstance(new_X_trains[2],pd.DataFrame) and isinstance(new_X_trains[3],pd.DataFrame)


  return new_X_trains


In [62]:
def PBC4cip_classifier_SelectionAlg(csv_file, t_count):
  _, y_train, _, _ = load_train_test_data(csv_file)
  classifier = PBC4cip(filtering=True,tree_count = t_count)
  all_patterns = []
  X_train_selectalgor = attribute_selection_df_generator(csv_file)
  for i in range(len(X_train_selectalgor)):
    all_patterns.append(classifier.fit(X_train_selectalgor[i], y_train))
  return all_patterns

In [63]:
def test_PBC4cip_classifier_SelectionAlg():
  all_patterns = PBC4cip_classifier_SelectionAlg('universities.csv', t_count=10)
  assert len(all_patterns) == 4
  assert isinstance(all_patterns[0],list) and isinstance(all_patterns[1],list) and isinstance(all_patterns[2],list) and isinstance(all_patterns[3],list)
  assert len(all_patterns[0]) >= 200 and len(all_patterns[1]) >= 200 and len(all_patterns[2]) >= 200 and len(all_patterns[3]) >= 200
  assert isinstance(all_patterns[0][0].Items,list) and isinstance(all_patterns[0][0].Supports,list)
  assert isinstance(allp[0][0].Supports[0], float)

# test_PBC4cip_classifier_SelectionAlg()

In [64]:
def get_patterns_SelectionAlg(csv_file, t_count):
  all_patterns = PBC4cip_classifier_SelectionAlg(csv_file, t_count)
  get_patterns = []
  for patterns in all_patterns:
    get_patterns.append(evaluate_patterns(patterns, diff = 0.3))
    
  return get_patterns

In [65]:
def test_get_patterns_SelectionAlg():
  patterns = get_patterns_SelectionAlg('universities.csv', t_count=10)
  assert len(patterns) == 4
  assert len(patterns[0]) == 4
  assert len(patterns[3]) == 1
  patt = []
  for pattern in patterns[0]:
    patt.append(str(pattern.Items[0]))
  assert 'cit2017 > 75794.500' in patt
  patt = []
  for pattern in patterns[2]:
    patt.append(str(pattern.Items[0]))
  assert 'h5Index > 167.500' in patt

## Results

### 10,000 trees patterns results

In [38]:
get_patterns_PBC4cip_classifier('Universities_RQS_Update.csv', t_count = 10000)

KeyboardInterrupt: 

### Random attribute selection results

In [ ]:
get_patterns_RandFeat('Universities_RQS_Update.csv', t_count = 10000)

### Selection algorithm results

In [66]:
get_patterns_SelectionAlg('Universities_RQS_Update.csv', t_count = 10000)

KeyboardInterrupt: 

### Max Depth results

In [ ]:
# MaxDepth = 2

In [ ]:
# MaxDepth = 3

In [ ]:
# MaxDepth = 4

In [ ]:
# Maxdepth = 5

### Min Object by leaf

In [ ]:
# MinObj = 3

In [ ]:
# MinObj = 4

In [ ]:
# MinObj = 5

In [ ]:
# MinObj = 10